# importing all libraries

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout, Flatten, TimeDistributed, LSTM, Input, BatchNormalization, Conv2D, MaxPooling2D, Reshape, Conv1D, GlobalAveragePooling1D, MaxPooling1D, Lambda
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.losses import sparse_categorical_crossentropy
from keras.losses import categorical_crossentropy
import tensorflow_hub as hub
from PIL import Image
import gzip
from nltk.translate.bleu_score import corpus_bleu
from keras.preprocessing.sequence import pad_sequences
import spacy
import h5py
import os
import cv2
import pickle
import re
import shutil
import glob
import gzip
%matplotlib inline

In [3]:
import time

# creating path variables

In [4]:
path_gloss_df = '/content/drive/MyDrive/Colab Notebooks/gloss_vector_dataframe.zip'
path_embedding_images = '/content/drive/MyDrive/Colab Notebooks/emb_comp_i3d_train_zero.h5'
path_gloss_files = '/content/drive/MyDrive/Colab Notebooks/phoenix14t.pami0.train.annotations_only.gzip'

In [5]:
path_dev_gloss = '/content/drive/MyDrive/Colab Notebooks/phoenix14t.pami0.dev.annotations_only.gzip'
path_test_gloss = '/content/drive/MyDrive/Colab Notebooks/phoenix14t.pami0.test.annotations_only.gzip'

In [6]:
path_embedding_dev = '/content/drive/MyDrive/Colab Notebooks/emb_comp_i3d_dev_zero.h5'
path_embedding_test = '/content/drive/MyDrive/Colab Notebooks/emb_comp_i3d_test_zero.h5'

In [7]:
dev_img_embedding = "\\emb_comp_i3d_dev_zero.h5"
test_img_embedding = "\\emb_comp_i3d_test_zero.h5"

# loading annotations in a dataframes

In [8]:
with gzip.open(path_gloss_files,'rb') as f:
  annotation_gloss = pickle.load(f)

In [9]:
with gzip.open(path_dev_gloss,'rb') as f:
  annotation_dev = pickle.load(f)

In [10]:
with gzip.open(path_test_gloss,'rb')as f :
  annotation_test = pickle.load(f)

In [11]:
annotation_gloss = pd.DataFrame(annotation_gloss)

In [12]:
annotation_gloss_dev = pd.DataFrame(annotation_dev)



In [13]:
annotation_gloss_test = pd.DataFrame(annotation_test)

In [14]:
annotation_gloss.head()

,name,signer,gloss,text
0,train/11August_2010_Wednesday_tagesschau-1,Signer08,JETZT WETTER MORGEN DONNERSTAG ZWOELF FEBRUAR,und nun die wettervorhersage für morgen donner...
1,train/11August_2010_Wednesday_tagesschau-4,Signer08,ORT REGEN DURCH REGEN KOENNEN UEBERSCHWEMMUNG ...,mancherorts regnet es auch länger und ergiebig...
2,train/11August_2010_Wednesday_tagesschau-5,Signer08,NORDWEST HEUTE NACHT TROCKEN BLEIBEN SUEDWEST ...,im nordwesten bleibt es heute nacht meist troc...
3,train/11August_2010_Wednesday_tagesschau-6,Signer08,TAGSUEBER OFT REGEN GEWITTER KOENNEN MANCHMAL ...,auch am tag gibt es verbreitet zum teil kräfti...
4,train/11August_2010_Wednesday_tagesschau-7,Signer08,WOLKE LOCH SPEZIELL NORDWEST,größere wolkenlücken finden sich vor allem im ...


In [15]:
annotation_gloss_dev.head()

,name,signer,gloss,text
0,dev/11August_2010_Wednesday_tagesschau-2,Signer08,DRUCK TIEF KOMMEN,tiefer luftdruck bestimmt in den nächsten tage...
1,dev/11August_2010_Wednesday_tagesschau-3,Signer08,ES-BEDEUTET VIEL WOLKE UND KOENNEN REGEN GEWIT...,das bedeutet viele wolken und immer wieder zum...
2,dev/11August_2010_Wednesday_tagesschau-8,Signer08,WIND MAESSIG SCHWACH REGION WENN GEWITTER WIND...,meist weht nur ein schwacher wind aus untersch...
3,dev/25October_2010_Monday_tagesschau-22,Signer01,MITTWOCH REGEN KOENNEN NORDWEST WAHRSCHEINLICH...,am mittwoch hier und da nieselregen in der nor...
4,dev/05May_2011_Thursday_tagesschau-25,Signer08,JETZT WETTER WIE-AUSSEHEN MORGEN FREITAG SECHS...,und nun die wettervorhersage für morgen freita...


In [16]:
annotation_gloss_test.head()

,name,signer,gloss,text
0,test/25October_2010_Monday_tagesschau-17,Signer01,REGEN SCHNEE REGION VERSCHWINDEN NORD REGEN KO...,regen und schnee lassen an den alpen in der na...
1,test/25October_2010_Monday_tagesschau-24,Signer01,DONNERSTAG NORDWEST REGEN REGION SONNE WOLKE W...,am donnerstag regen in der nordhälfte in der s...
2,test/15December_2010_Wednesday_tagesschau-37,Signer05,KRAEFTIG AB MORGEN FRUEH MEISTENS SCHNEE SCHNE...,vom nordmeer zieht ein kräftiges tief heran un...
3,test/10March_2011_Thursday_heute-58,Signer01,WOCHENENDE SONNE SAMSTAG SCHOEN TEMPERATUR BIS...,sonnig geht es auch ins wochenende samstag ein...
4,test/14August_2009_Friday_tagesschau-62,Signer05,DEUTSCH LAND MORGEN HOCH DRUCK KOMMEN WOLKE AU...,deutschland liegt morgen unter hochdruckeinflu...


In [17]:
# we can use values attribute to directly create a list of all sentences of a particular column
#annotation_gloss_test["text"].values

# preprocessing the vocabulary

In [18]:
for i in range(len(annotation_gloss)):
  annotation_gloss.iloc[i,2] = 'startseq '+annotation_gloss.iloc[i,2].lower() + ' endseq'
  annotation_gloss.iloc[i,3] = 'startseq '+annotation_gloss.iloc[i,3].lower() + ' endseq'
  annotation_gloss.iloc[i,3] = annotation_gloss.iloc[i,3].replace(' .','')
  #annotation_gloss.iloc[i,3] =annotation_gloss.iloc[i,3][-2].replace(' ','')
 

In [19]:
for i in range(len(annotation_gloss_dev)):
  annotation_gloss_dev.iloc[i,2] = 'startseq '+annotation_gloss_dev.iloc[i,2].lower() + ' endseq'
  annotation_gloss_dev.iloc[i,3] = 'startseq '+annotation_gloss_dev.iloc[i,3].lower() + ' endseq'
  annotation_gloss_dev.iloc[i,3] = annotation_gloss_dev.iloc[i,3].replace(' .','')

In [20]:
for i in range(len(annotation_gloss_test)):
  annotation_gloss_test.iloc[i,2] = 'startseq '+annotation_gloss_test.iloc[i,2].lower() + ' endseq'
  annotation_gloss_test.iloc[i,3] = 'startseq '+annotation_gloss_test.iloc[i,3].lower() + ' endseq'
  annotation_gloss_test.iloc[i,3] = annotation_gloss_test.iloc[i,3].replace(' .','')

# creating a vocabulary of texts

In [21]:
vocabulary = []
for txt in annotation_gloss.text.values:
    vocabulary.extend(txt.split())
for txt in annotation_gloss.gloss.values:
    vocabulary.extend(txt.split())
    
for txt in annotation_gloss_dev.text.values:
    vocabulary.extend(txt.split())
for txt in annotation_gloss_dev.gloss.values:
    vocabulary.extend(txt.split())
    
for txt in annotation_gloss_test.text.values:
    vocabulary.extend(txt.split())
for txt in annotation_gloss_test.gloss.values:
    vocabulary.extend(txt.split())
print('Vocabulary Size: %d' % len(set(vocabulary)))

Vocabulary Size: 3606


# reading images embedding for train test and dev

In [22]:
#reading images feature vectors
with h5py.File(path_embedding_images,'r')as ab:
    ind = ab.keys()
    print(ind)
    data_train=ab['x_1'][()]

<KeysViewHDF5 ['x_1']>


In [23]:
#reading images feature vectors
with h5py.File(path_embedding_dev,'r')as ab:
    ind = ab.keys()
    print(ind)
    data_dev=ab['x_1'][()]

<KeysViewHDF5 ['x_1']>


In [24]:
#reading images feature vectors
with h5py.File(path_embedding_test,'r')as ab:
    ind = ab.keys()
    print(ind)
    data_test=ab['x_1'][()]

<KeysViewHDF5 ['x_1']>


In [25]:
data_train.shape

(7096, 512)

In [26]:
data_dev.shape

(519, 512)

In [27]:
data_test.shape

(642, 512)

# creating tokenizer dataset


In [28]:
gloss_token = []
text_token = []
for i in range(len(annotation_gloss)):
  gloss_token.append(annotation_gloss.iloc[i,2])
  text_token.append(annotation_gloss.iloc[i,3])

In [29]:
gloss_token_dev = []
text_token_dev = []
for i in range(len(annotation_gloss_dev)):
  gloss_token_dev.append(annotation_gloss_dev.iloc[i,2])
  text_token_dev.append(annotation_gloss_dev.iloc[i,3])

In [30]:
gloss_token_test = []
text_token_test = []
for i in range(len(annotation_gloss_test)):
  gloss_token_test.append(annotation_gloss_test.iloc[i,2])
  text_token_test.append(annotation_gloss_test.iloc[i,3])

# creating tokenizer

In [31]:
def create_tokenizer(vocabulary):
    tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=3606,
                                                 oov_token="<unk>",
                                                 filters='!"#$%&()*+.,-/:;=?@[\]^_`{|}~ ')
    tokenizer.fit_on_texts(vocabulary)
    tokenizer.word_index['<pad>'] = 0
    tokenizer.index_word[0] = '<pad>'
    return tokenizer

In [32]:
tokenizer = create_tokenizer(vocabulary)

In [33]:
vocab_size = len(tokenizer.word_index)+1
print("vocalb size",vocab_size)

vocalb size 3436


In [34]:
tokenizer.index_word[3]

'endseq'

In [35]:
def encode_sequences(tokenizer, length, lines):
    # integer encode sequences
    X = tokenizer.texts_to_sequences(lines)
    # pad sequences with 0 values
    X = pad_sequences(X, maxlen=length, padding='post')
    return X

# tokenizing train,test,dev glosses

In [36]:
train_y = encode_sequences(tokenizer,35,gloss_token)
dev_y = encode_sequences(tokenizer,35,gloss_token_dev)
test_y = encode_sequences(tokenizer,35,gloss_token_test)

In [37]:
train_y[0]

array([  2,  72,  31,   9,  70, 262, 327,   3,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0], dtype=int32)

# hyperparameters

In [ ]:
# IGNORE THIS REPRESENTATION


In [38]:
BATCH_SIZE = 128
BUFFER_SIZE = 7096
embedding_dim = 256
units = 512
vocab_size = len(tokenizer.word_index) + 1
num_steps = 7096 // BATCH_SIZE
features_shape = 512
attention_features_shape = 35

In [ ]:
# data_train.shape

In [ ]:
# for i in train_y[2]:
# print(tokenizer.index_word[i])

In [39]:
dataset = tf.data.Dataset.from_tensor_slices((data_train, train_y))
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)

# dataset shape = ((128,512),(128,35))
# dataset shape = ((batchsize,feature size),(batch size ,sentence size))

In [ ]:
for (batch, (img_tensor, target)) in enumerate(dataset):
    print("batch",batch)
    
    print("img_tensor",img_tensor)
    print("img tensrog shape",img_tensor.shape)
    print("target",target)
    print("tartghet shape",target.shape)
    break

# CREATING NEW REPRESENATION


In [ ]:
#pip install tensorflow-lattice

In [45]:
import tensorflow_lattice.layers as tfl

ModuleNotFoundError: ignored

In [46]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

In [47]:
data_train2= data_train.copy()
data_dev2 = data_dev.copy()
data_test2 = data_test.copy()

In [48]:
class FeatureEmbedder(nn.Module):
    
    def __init__(self, d_feat, d_model):
        super(FeatureEmbedder, self).__init__()
        self.d_model = d_model
        self.embedder = nn.Linear(d_feat, d_model)
        
    def forward(self, x): # x - tokens (B, seq_len, d_feat)
        x = self.embedder(x)
        x = x * np.sqrt(self.d_model)
        
        return x # (B, seq_len, d_model)

In [49]:
data_train2 = torch.from_numpy(data_train2)

In [50]:
data_train2 = data_train2.to(torch.float)

In [51]:
data_train2 = data_train2.reshape(7096,512,1)

In [52]:
features = FeatureEmbedder(1,32)

In [53]:
temp_data = features(data_train2)

In [54]:
temp_data.shape

torch.Size([7096, 512, 32])

In [55]:
BATCH_SIZE = 64
BUFFER_SIZE = 1000
num_steps = 7096 // BATCH_SIZE

In [56]:
train_temp = temp_data.detach().numpy()

In [57]:
data_train_temp = tf.cast(train_temp,dtype = tf.float32)

In [58]:
data_train_temp.shape

TensorShape([7096, 512, 32])

In [59]:
def map_func(img_name, cap):
   
   return img_name, cap



In [60]:
dataset = tf.data.Dataset.from_tensor_slices((data_train_temp, train_y))
dataset = dataset.map(lambda item1, item2: tf.numpy_function(map_func, [item1, item2], [tf.float32, tf.int32]),num_parallel_calls=tf.data.experimental.AUTOTUNE)
dataset = dataset.batch(BATCH_SIZE)
dataset = dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

In [ ]:
for (batch,(img,tar)) in enumerate(dataset.as_numpy_iterator()):
    print(batch)
    print(img)
    print("********")
    print(tar)
    break
    

# creating model for decoder

In [62]:

class BahdanauAttention(tf.keras.Model):
  def __init__(self, units):
    super(BahdanauAttention, self).__init__()
    self.W1 = tf.keras.layers.Dense(units)
    self.W2 = tf.keras.layers.Dense(units)
    self.V = tf.keras.layers.Dense(1)

  def call(self, features, hidden):
    # features(CNN_encoder output) shape == (batch_size, 64, embedding_dim)
    print("features shape in  bhadanu attention",features.shape)

    # hidden shape == (batch_size, hidden_size)
    # hidden_with_time_axis shape == (batch_size, 1, hidden_size)
    hidden_with_time_axis = tf.expand_dims(hidden, 1)
    print("hidden sahpe in bhadanau",hidden_with_time_axis.shape)

    # attention_hidden_layer shape == (batch_size, 64, units)
    attention_hidden_layer = (tf.nn.tanh(self.W1(features) +
                                         self.W2(hidden_with_time_axis)))

    # score shape == (batch_size, 64, 1)
    # This gives you an unnormalized score for each image feature.
    score = self.V(attention_hidden_layer)

    # attention_weights shape == (batch_size, 64, 1)
    attention_weights = tf.nn.softmax(score, axis=1)

    # context_vector shape after sum == (batch_size, hidden_size)
    context_vector = attention_weights * features
    context_vector = tf.reduce_sum(context_vector, axis=1)

    return context_vector, attention_weights





In [ ]:
# class CNN_Encoder(tf.keras.Model):
#     # Since you have already extracted the features and dumped it
#     # This encoder passes those features through a Fully connected layer
#     def __init__(self, embedding_dim):
#         super(CNN_Encoder, self).__init__()
#         # shape after fc == (batch_size, 64, embedding_dim)
#         self.fc = tf.keras.layers.Dense(embedding_dim)

#     def call(self, x):
#         x = self.fc(x)
#         x = tf.nn.relu(x)
#         return x

In [76]:
class RNN_Decoder(tf.keras.Model):
  def __init__(self, embedding_dim, units, vocab_size):
    super(RNN_Decoder, self).__init__()
    self.units = units

    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(self.units,
                                   return_sequences=True,
                                   return_state=True,
                                   recurrent_initializer='glorot_uniform')
    self.fc1 = tf.keras.layers.Dense(self.units)
    self.fc2 = tf.keras.layers.Dense(vocab_size)

    self.attention = BahdanauAttention(self.units)

  def call(self, x, features, hidden):
    # defining attention as a separate model
    print("features shape in decoder part",features.shape)
    context_vector, attention_weights = self.attention(features, hidden)
    print("contxt vaext",context_vector.shape)

    # x shape after passing through embedding == (batch_size, 1, embedding_dim)
    x = self.embedding(x)

    # x shape after concatenation == (batch_size, 1, embedding_dim + hidden_size)
    x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1)

    # passing the concatenated vector to the GRU
    output, state = self.gru(x)

    # shape == (batch_size, max_length, hidden_size)
    x = self.fc1(output)

    # x shape == (batch_size * max_length, hidden_size)
    x = tf.reshape(x, (-1, x.shape[2]))

    # output shape == (batch_size * max_length, vocab)
    x = self.fc2(x)

    return x, state, attention_weights

  def reset_state(self, batch_size):
    return tf.zeros((batch_size, self.units))


In [80]:

embedding_dim = 256
units = 512
vocab_size = vocab_size
num_steps = 7096 // BATCH_SIZE
# Shape of the vector extracted from InceptionV3 is (64, 2048)
# These two variables represent that vector shape
features_shape = 512
attention_features_shape = 32

#encoder = CNN_Encoder(embedding_dim)
decoder = RNN_Decoder(embedding_dim, units, vocab_size)

optimizer = tf.keras.optimizers.Adam()
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

def loss_function(real, pred):
  mask = tf.math.logical_not(tf.math.equal(real, 0))
  loss_ = loss_object(real, pred)

  mask = tf.cast(mask, dtype=loss_.dtype)
  loss_ *= mask

  return tf.reduce_mean(loss_)

loss_plot = []
@tf.function
def train_step(img_tensor, target):
  loss = 0

  # initializing the hidden state for each batch
  # because the captions are not related from image to image
  hidden = decoder.reset_state(batch_size=target.shape[0])

  dec_input = tf.expand_dims([tokenizer.word_index['startseq']] * target.shape[0], 1)

  with tf.GradientTape() as tape:
      features = img_tensor

      for i in range(1, target.shape[1]):
          # passing the features through the decoder
          predictions, hidden, _ = decoder(dec_input, features, hidden)

          loss += loss_function(target[:, i], predictions)

          # using teacher forcing
          dec_input = tf.expand_dims(target[:, i], 1)

  total_loss = (loss / int(target.shape[1]))

  trainable_variables = decoder.trainable_variables

  gradients = tape.gradient(loss, trainable_variables)

  optimizer.apply_gradients(zip(gradients, trainable_variables))

  return loss, total_loss



In [82]:
import time
start_epoch = 0
EPOCHS = 200

for epoch in range(start_epoch, EPOCHS):
    start = time.time()
    total_loss = 0

    for (batch, (img_tensor, target)) in enumerate(dataset.as_numpy_iterator()):
        batch_loss, t_loss = train_step(img_tensor, target)
        total_loss += t_loss

        if batch % 100 == 0:
            average_batch_loss = batch_loss.numpy()/int(target.shape[1])
            print(f'Epoch {epoch+1} Batch {batch} Loss {average_batch_loss:.4f}')
    # storing the epoch end loss value to plot later
    loss_plot.append(total_loss / num_steps)

    print(f'Epoch {epoch+1} Loss {total_loss/num_steps:.6f}')
    print(f'Time taken for 1 epoch {time.time()-start:.2f} sec\n')


Epoch 1 Batch 0 Loss 0.9299
Epoch 1 Batch 100 Loss 0.9330
Epoch 1 Loss 0.919444
Time taken for 1 epoch 26.93 sec

Epoch 2 Batch 0 Loss 0.9280
Epoch 2 Batch 100 Loss 0.9296
Epoch 2 Loss 0.917306
Time taken for 1 epoch 27.03 sec

Epoch 3 Batch 0 Loss 0.9260
Epoch 3 Batch 100 Loss 0.9320
Epoch 3 Loss 0.917526
Time taken for 1 epoch 26.93 sec

Epoch 4 Batch 0 Loss 0.9282
Epoch 4 Batch 100 Loss 0.9261
Epoch 4 Loss 0.914294
Time taken for 1 epoch 26.88 sec

Epoch 5 Batch 0 Loss 0.9246
Epoch 5 Batch 100 Loss 0.9257
Epoch 5 Loss 0.912325
Time taken for 1 epoch 27.00 sec

Epoch 6 Batch 0 Loss 0.9223
Epoch 6 Batch 100 Loss 0.9242
Epoch 6 Loss 0.911207
Time taken for 1 epoch 26.94 sec

Epoch 7 Batch 0 Loss 0.9194
Epoch 7 Batch 100 Loss 0.9278
Epoch 7 Loss 0.910133
Time taken for 1 epoch 26.93 sec

Epoch 8 Batch 0 Loss 0.9145
Epoch 8 Batch 100 Loss 0.9206
Epoch 8 Loss 0.907604
Time taken for 1 epoch 27.06 sec

Epoch 9 Batch 0 Loss 0.9178
Epoch 9 Batch 100 Loss 0.9151
Epoch 9 Loss 0.904945
Time tak

KeyboardInterrupt: ignored

 # loss

In [83]:
data_dev2 = data_dev.copy()

In [84]:
data_dev2 = torch.from_numpy(data_dev2)

In [85]:
data_dev2 = data_dev2.to(torch.float)

In [86]:
data_dev2 = data_dev2.reshape(519,512,1)

In [87]:
features_dev = FeatureEmbedder(1,32)

In [88]:
temp_data_dev = features_dev(data_dev2)

In [89]:
BATCH_SIZE = 1
BUFFER_SIZE = 1000
num_steps = 519 // BATCH_SIZE

In [90]:
temp_dev = temp_data_dev.detach().numpy()

In [91]:
data_dev_temp = tf.cast(temp_dev,dtype = tf.float32)

In [92]:
#data_dev_temp = data_dev_temp.reshape(519,32,512)

In [93]:
dataset = tf.data.Dataset.from_tensor_slices((data_dev_temp, dev_y))
dataset = dataset.map(lambda item1, item2: tf.numpy_function(map_func, [item1, item2], [tf.float32, tf.int32]),num_parallel_calls=tf.data.experimental.AUTOTUNE)


In [94]:
# dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
# dataset = dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
#dataset = dataset.batch(64)
dataset = dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

In [ ]:
tokenizer.word_index['startseq']

In [97]:
#!pip install jiwer

     |████████████████████████████████| 51kB 4.1MB/s 
  Created wheel for python-Levenshtein: filename=python_Levenshtein-0.12.2-cp37-cp37m-linux_x86_64.whl size=149817 sha256=38bbca7af097ade880fdf646866deb69a19db9b06af4ae417f7b9afe60cf782b
  Stored in directory: /root/.cache/pip/wheels/b3/26/73/4b48503bac73f01cf18e52cd250947049a7f339e940c5df8fc
Successfully built python-Levenshtein


In [98]:
import jiwer

In [99]:
from nltk.translate.bleu_score import SmoothingFunction
smoothie = SmoothingFunction().method4

# evalute

In [100]:

def evaluate(vid):
   #attention_plot = np.zeros((max_length, attention_features_shape))
   #print("vid shape",vid.shape)

   hidden = decoder.reset_state(batch_size=1)
   temp_input = tf.expand_dims(vid, 0)
   max_length = 35
   #img_tensor_val = image_features_extract_model(temp_input)
   #img_tensor_val = tf.reshape(img_tensor_val, (img_tensor_val.shape[0], -1, img_tensor_val.shape[3])

   #features = encoder(img_tensor_val)
   #print("features shape",vid.shape)
   features = temp_input
   #features = vid
   #print("feature shape",features.shape)
   dec_input = tf.expand_dims([tokenizer.word_index['startseq']], 1)
   result = []

   for i in range(max_length):
       #print("dec_input" ,dec_input.shape)
       #print("features",features.shape)
       #print("hidden",hidden.shape)
       predictions, hidden, attention_weights = decoder(dec_input, features, hidden)
       #attention_plot[i] = tf.reshape(attention_weights, (-1, )).numpy()
       predicted_id = tf.argmax(predictions[0]).numpy()
       result.append(tokenizer.index_word[predicted_id])
       #print("current predicted word",tokenizer.index_word[predicted_id])

       if tokenizer.index_word[predicted_id] == 'endseq':
           return result

       dec_input = tf.expand_dims([predicted_id], 0)
   #attention_plot = attention_plot[:len(result), :]

   #return result, attention_plot
   return result

In [102]:
actual,predicted = list(),list()
for (batch, (img_tensor, target)) in enumerate(dataset.as_numpy_iterator()):
  for i in range(len(img_tensor)):
    image = img_tensor[i]
    result = evaluate(image)
    real_caption = text_token_dev[i].split(' ')
    for j in real_caption:
      if((j=='startseq')or (j=="endseq")):
        real_caption.remove(j)
    first = ' '.join(j for j in real_caption)
    for j in result:
      if j=="<unk>":
        result.remove(j)

    # for j in real_caption:
    #   if (j=="<unk>") or(j=="endseq") :
    #     real_caption.remove(j)
    #first = real_caption.split(' ', 1)[1]
    result_join = ' '.join(result)
    result_final = result_join.rsplit(' ', 1)[0]
    print("predicted sentence",result_final)
    print("actual sentence",first)
    actual.append(first)
    predicted.append(result_final)
    # print('BLEU-1: %f' % corpus_bleu([first],[result_final], weights=(1.0, 0, 0, 0),smoothing_function=smoothie))
    # print('BLEU-2: %f' % corpus_bleu([first],[result_final], weights=(0.5, 0.5, 0, 0),smoothing_function=smoothie))
    # print('BLEU-3: %f' % corpus_bleu([first],[result_final], weights=(0.3, 0.3, 0.3, 0),smoothing_function=smoothie))
    # print('BLEU-4: %f' % corpus_bleu([first],[result_final], weights=(0.25, 0.25, 0.25, 0.25),smoothing_function=smoothie))
    print("word error rate is",jiwer.wer(first,result_final))
      

print('BLEU-1: %f' % corpus_bleu(actual,predicted, weights=(1.0, 0, 0, 0),smoothing_function=smoothie))
print('BLEU-2: %f' % corpus_bleu(actual,predicted, weights=(0.5, 0.5, 0, 0),smoothing_function=smoothie))
print('BLEU-3: %f' % corpus_bleu(actual,predicted, weights=(0.3, 0.3, 0.3, 0),smoothing_function=smoothie))
print('BLEU-4: %f' % corpus_bleu(actual,predicted, weights=(0.25, 0.25, 0.25, 0.25),smoothing_function=smoothie))

Streaming output truncated to the last 5000 lines.
features shape in  bhadanu attention (1, 32)
hidden sahpe in bhadanau (1, 1, 512)
contxt vaext (1, 32)
features shape in decoder part (1, 32)
features shape in  bhadanu attention (1, 32)
hidden sahpe in bhadanau (1, 1, 512)
contxt vaext (1, 32)
features shape in decoder part (1, 32)
features shape in  bhadanu attention (1, 32)
hidden sahpe in bhadanau (1, 1, 512)
contxt vaext (1, 32)
features shape in decoder part (1, 32)
features shape in  bhadanu attention (1, 32)
hidden sahpe in bhadanau (1, 1, 512)
contxt vaext (1, 32)
features shape in decoder part (1, 32)
features shape in  bhadanu attention (1, 32)
hidden sahpe in bhadanau (1, 1, 512)
contxt vaext (1, 32)
features shape in decoder part (1, 32)
features shape in  bhadanu attention (1, 32)
hidden sahpe in bhadanau (1, 1, 512)
contxt vaext (1, 32)
predicted sentence atlantik auftauchen hochwasser eis kratzen ab mittag eigentlich west ohne wolke himmel mainz zeigen bildschirm
actual

KeyboardInterrupt: ignored